In [2]:
library(tidyverse)

Regular expressions are the default pattern engine in stringr. That means when you use a pattern matching function with a bare string, it’s equivalent to wrapping it in a call to `regex()`:

In [3]:
'soso-777-' %>% str_extract('[0-9]+')

[1] "777"

In [4]:
'soso-777-' %>% str_extract(regex('[0-9]+'))

[1] "777"

You will need to use `regex()` explicitly if you want to override the default options, as you’ll see in examples below.

# Basic Matches

In [5]:
x <- c("apple", "banana", "pear")
x

[1] "apple"  "banana" "pear"

The simplest patterns match exact strings:

In [6]:
x %>% str_extract('an')

[1] NA   "an" NA

case-insensitive matching using `ignore_case = TRUE`:

In [8]:
x %>% str_extract(regex('AN', ignore_case = T))

[1] NA   "an" NA

The next step up in complexity is `.`, which matches any character except a newline:

In [10]:
x %>% str_extract('.a.')

[1] NA    "ban" "ear"

You can allow `.` to match everything, including `\n`, by setting `dotall = TRUE`:

In [13]:
"\nX\n" %>% str_detect('...')

"\nX\n" %>% str_detect(regex('...', dotall = T))

[1] FALSE

[1] TRUE

# Escaping

using `\\`

for `\`, escape by using `\\\\`

In [19]:
# detect character '+'
'1+2' %>% str_detect('\\+')

[1] TRUE

In [23]:
# detect digit \d
'a3' %>% str_detect('\\d')

[1] TRUE

In [25]:
x <- "a\\b"
x

[1] "a\\b"

In [26]:
x %>% str_extract('\\\\')

[1] "\\"

An alternative quoting mechanism is `\Q...\E`: all the characters in `...` are treated as exact matches. This is useful if you want to exactly match user input as part of a regular expression.

In [27]:
x <- c("a.b.c.d", "aeb")
starts_with <- "a.b"

str_detect(x, paste0("^", starts_with))
#> [1] TRUE TRUE
str_detect(x, paste0("^\\Q", starts_with, "\\E"))

[1] TRUE TRUE

[1]  TRUE FALSE

In [31]:
input_user <- '.'

'1a2' %>% str_detect(input_user)

# Match the character '.'
'1a2' %>% str_detect(paste0('\\Q', input_user, '\\E'))

[1] TRUE

[1] FALSE

# Special characters

# Matching multiple characters

 `.` will match the component “a”, while `\X` will match the complete symbol:

In [57]:
x <- "a\u0301"
x

[1] "a´"

In [58]:
str_extract(x, ".")

[1] "a"

In [60]:
str_extract(x, "\\X")

[1] "a´"

There are five other escaped pairs that match narrower classes of characters:

* `\d`: matches any digit. The complement, `\D`, matches any character that is not a decimal digit.
* `\s`: matches any whitespace. This includes tabs, newlines, form feeds, and any character in the Unicode Z Category (which includes a variety of space characters and other separators.). The complement, `\S`, matches any non-whitespace character.
* `\p{property name}` matches any character with specific unicode property, like `\p{Uppercase}` or `\p{Diacritic}`.   
The complement, `\P{property name}`, matches all characters without the property. A complete list of unicode properties can be found at **http://www.unicode.org/reports/tr44/#Property_Index**.
* `\w` matches any “word” character, which includes alphabetic characters, marks and decimal numbers. The complement, \W, matches any non-word character.
* `\b` matches word boundaries, the transition between word and non-word characters. \B matches the opposite: boundaries that have either both word or non-word characters on either side.

In [62]:
str_replace_all("The quick brown fox", "\\b", "_")

str_replace_all("The quick brown fox", "\\B", "_")


[1] "_The_ _quick_ _brown_ _fox_"

[1] "T_h_e q_u_i_c_k b_r_o_w_n f_o_x"

You can also create your own character classes using `[]`:

* `[abc]`: matches a, b, or c.
* `[a-z]`: matches every character between a and z (in Unicode code point order).
* `[^abc]`: matches anything except a, b, or c.
* `[\^\-]`: matches ^ or -.

There are a number of pre-built classes that you can use inside []:

* `[:punct:]`: punctuation.
* `[:alpha:]`: letters.
* `[:lower:]`: lowercase letters.
* `[:upper:]`: upperclass letters.
* `[:digit:]`: digits.
* `[:xdigit:]`: hex digits.
* `[:alnum:]`: letters and numbers.
* `[:cntrl:]`: control characters.
* `[:graph:]`: letters, numbers, and punctuation.
* `[:print:]`: letters, numbers, punctuation, and whitespace.
* `[:space:]`: space characters (basically equivalent to \s).
* `[:blank:]`: space and tab.

These all go inside the `[]` for character classes, i.e. `[[:digit:]AX]` matches all digits, A, and X.

# Alternation

`|` is the **alternation** operator, which will pick between one or more possible matches.

In [33]:
c('abc', 'def', 'abcd', 'ghi') %>% str_detect('abc|def')

[1]  TRUE  TRUE  TRUE FALSE

# Grouping

You can use parentheses to override the default precedence rules:

In [36]:
c('grey', 'gray') %>% str_extract_all('gre|ay')

[[1]]
[1] "gre"

[[2]]
[1] "ay"

In [38]:
c('grey, gray') %>% str_extract_all('gr(e|a)y')

[[1]]
[1] "grey" "gray"

Parenthesis also define “groups” that you can refer to with backreferences, like `\1`, `\2` etc, and can be extracted with `str_match()`. For example, the following regular expression finds all string that have a repeated pair of letters:

In [41]:
c('moon light apple', 'how do you like it') %>% str_extract_all('(.)\\1')

[[1]]
[1] "oo" "pp"

[[2]]
character(0)

You can use `(?:...)`, the non-grouping parentheses, to control precedence but not capture the match in a group. This is slightly more efficient than capturing parentheses.

In [42]:
str_match(c("grey", "gray"), "gr(e|a)y")

grey,e
gray,a


In [43]:
str_match(c("grey", "gray"), "gr(?:e|a)y")

grey
gray


# Anchors

By default, regular expressions will match any part of a string. It’s often useful to anchor the regular expression so that it matches from the start or end of the string:

* `^` matches the start of string.
* `$` matches the end of the string.

In [44]:
x <- c("apple", "banana", "pear")

In [45]:
# filter strings start with 'a'
x %>% str_subset('^a')

[1] "apple"

In [46]:
# filter strings ends with 'a'
x %>% str_subset('a$')

[1] "banana"

For multiline strings, you can use `regex(multiline = TRUE)`. This changes the behaviour of `^` and `$`, and introduces three new operators:

* `^` now matches the start of each line.

* `$` now matches the end of each line.

* `\A` matches the start of the input.

* `\z` matches the end of the input.

* `\Z` matches the end of the input, but before the final line terminator, if it exists.

In [47]:
x <- "Line 1\nLine 2\nLine 3\n"
cat(x)

Line 1
Line 2
Line 3


In [48]:
str_extract_all(x, "^Line..")

[[1]]
[1] "Line 1"

In [49]:
str_extract_all(x, regex('^Line..', multiline = T))

[[1]]
[1] "Line 1" "Line 2" "Line 3"

# Repetition

You can control how many times a pattern matches with the repetition operators:

*  `?`: 0 or 1.
*  `+`: 1 or more.
*  `*`: 0 or more.

Note that the precedence of these operators is high, so you can write: `colou?r` to match either American or British spellings. That means most uses will need parentheses, like `bana(na)+`.

You can also specify the number of matches precisely:

* `{n}`: exactly n
* `{n,}`: n or more
* `{n,m}`: between n and m

By default these matches are “greedy”: they will match the longest string possible. You can make them “lazy”, matching the shortest string possible by putting a ? after them:

* `??`: 0 or 1, prefer 0.
* `+?`: 1 or more, match as few times as possible.
* `*?`: 0 or more, match as few times as possible.
* `{n,}?`: n or more, match as few times as possible.
* `{n,m}?`: between n and m, , match as few times as possible, but at least n.

You can also make the matches possessive by putting a + after them, which means that if later parts of the match fail, the repetition will not be re-tried with a smaller number of characters. This is an advanced feature used to improve performance in worst-case scenarios (called “catastrophic backtracking”).

* `?+`: 0 or 1, possessive.
* `++`: 1 or more, possessive.
* `*+`: 0 or more, possessive.
* `{n}+`: exactly n, possessive.
* `{n,}+`: n or more, possessive.
* `{n,m}+`: between n and m, possessive.

A related concept is the **atomic-match** parenthesis, `(?>...)`. If a later match fails and the engine needs to back-track, an atomic match is kept as is: it succeeds or fails as a whole. Compare the following two regular expressions:

In [50]:
str_detect("ABC", "(?>A|.B)C")


str_detect("ABC", "(?:A|.B)C")

[1] FALSE

[1] TRUE

The atomic match fails because it matches A, and then the next character is a C so it fails. The regular match succeeds because it matches A, but then C doesn’t match, so it back-tracks and tries B instead.

# Look arounds

These assertions look ahead or behind the current match without “consuming” any characters (i.e. changing the input position).

* `(?=...)`: positive look-ahead assertion. Matches if ... matches at the current input.

* `(?!...)`: negative look-ahead assertion. Matches if ... does not match at the current input.

* `(?<=...)`: positive look-behind assertion. Matches if ... matches text preceding the current position, with the last character of the match being the character just before the current position. Length must be bounded
(i.e. no * or +).

* `(?<!...)`: negative look-behind assertion. Matches if ... does not match text preceding the current position. Length must be bounded
(i.e. no * or +).

These are useful when you want to check that a pattern exists, but you don’t want to include it in the result:

In [53]:
x <- c("1 piece", "2 pieces", "3")

str_extract(x, "\\d+(?= pieces?)")

[1] "1" "2" NA

In [52]:
y <- c("100", "$400")
str_extract(y, "(?<=\\$)\\d+")

[1] NA    "400"

# Comments

There are two ways to include comments in a regular expression. The first is with `(?#...)`:

In [55]:
str_detect("xyz", "x(?#this is a comment)")

[1] TRUE

The second is to use `regex(comments = TRUE)`. This form ignores spaces and newlines, and anything everything after `#`. To match a literal space, you’ll need to escape it: `"\\ "`. This is a useful way of describing complex regular expressions:

In [56]:
phone <- regex("
  \\(?     # optional opening parens
  (\\d{3}) # area code
  [)- ]?   # optional closing parens, dash, or space
  (\\d{3}) # another three numbers
  [ -]?    # optional space or dash
  (\\d{3}) # three more numbers
  ", comments = TRUE)

str_match("514-791-8141", phone)

514-791-814,514,791,814


# Other uses of regular expression in R

There are two useful function in base R that also use regular expressions:

- **`apropos()`** searches all objects available from the global environment. This is useful if you can’t quite remember the name of the function.

In [3]:
apropos('replace')

[1] "%+replace%"       "replace"          "replace_na"       "setReplaceMethod"
[5] "str_replace"      "str_replace_all"  "str_replace_na"   "theme_replace"

- **`dir()`** lists all the files in a directory. The pattern argument takes a regular expression and only returns file names that match the pattern. For example, you can find all the R Markdown files in the current directory with:

In [8]:
dir(pattern = '.ipynb$')

[1] "Get started.ipynb"                   "Regular Expression.ipynb"           
[3] "The principle of broadcasting.ipynb"